# Final Code File is this one

Huzaifa Bin Tariq - 25133

Hazim Ghulam Farooq - 25148

Muhammad Wasay - 24497

In [ ]:
!pip install -q rank_bm25 faiss-cpu langchain transformers accelerate sentence-transformers pypdf langchain_community PyPDF2
!pip install chromadb
!pip install groq
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12

In [4]:
import time
import json
import numpy as np
from PyPDF2 import PdfReader
from sklearn.metrics.pairwise import cosine_similarity
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from groq import Groq
import os
from langchain_core.language_models import BaseLLM
from langchain_core.outputs import LLMResult
from typing import Any, List, Optional, Mapping, Dict
import chromadb
from pydantic import Field
from rouge_score import rouge_scorer
from sentence_transformers import CrossEncoder


In [5]:
from time import sleep

# Configure Groq (replace with your API key)
os.environ["GROQ_API_KEY"] = "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"
client = Groq()

# Questions to generate answers for
questions = [
    {
        "question": "Who is Adam (A.S)?",
        "difficulty": "easy"
    },
    {
        "question": "What is the significance of the 'forbidden tree' in the story of Adam (AS)?",
        "difficulty": "easy"
    },
    {
        "question": "Why did the Jews of Madinah oppose Islam?",
        "difficulty": "medium"
    },
    {
        "question": "What does the Qur'an say about the hypocrites (Munafiqun)?",
        "difficulty": "medium"
    },
    {
        "question": "What lessons does the story of Adam (AS) teach about free will and repentance?",
        "difficulty": "hard"
    },
    {
        "question": "How does the Qur'an use the Israelites' history as a warning for Muslims?",
        "difficulty": "hard"
    }
]

# System prompt to enforce Surah Al-Baqarah-only answers
system_prompt = """You are a Quranic tafsir specialist. Answer with:
1. **Exclusive focus**: Only use Surah Al-Baqarah (Chapter 2) as the source.
2. **Verse citations**: Always reference ayat numbers (e.g., 2:143).
3. **Classical tafsir**: Prioritize interpretations from Ibn Kathir/Tabari.
4. **Rejection clause**: If the question cannot be answered from Surah Al-Baqarah, state: "This is not addressed in Surah Al-Baqarah."

Example format:
Question: "What is riba?"
Answer: "Surah Al-Baqarah (2:275-279) prohibits riba (usury) and contrasts it with lawful trade..."
"""

# Generate answers using deepseek r1
ground_truth_qa = []

for q in questions:
    try:
        response = client.chat.completions.create(
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": q["question"]}
            ],
            model="llama3-70b-8192",  # More accurate than Mixtral for tafsir
            temperature=0.2,  # Lower = more factual
            max_tokens=600   # Allows detailed explanations with verses
        )

        answer = response.choices[0].message.content

        # Post-processing to ensure compliance
        if "Surah Al-Baqarah" not in answer and "2:" not in answer:
            answer = "[RAGAS FILTERED] This answer could not be verified from Surah Al-Baqarah. Original response: " + answer

        ground_truth_qa.append({
            "question": q["question"],
            "answer": answer,
            "difficulty": q["difficulty"]
        })

        print(f"Processed: {q['question']}")
        sleep(1.5)  # Avoid rate limits

    except Exception as e:
        print(f"Error on '{q['question']}': {str(e)}")
        ground_truth_qa.append({
            "question": q["question"],
            "answer": f"[ERROR] Failed to generate answer: {str(e)}",
            "difficulty": q["difficulty"]
        })

# Save results
with open("surah_baqarah_ground_truth.json", "w", encoding="utf-8") as f:
    json.dump(ground_truth_qa, f, ensure_ascii=False, indent=4)

print("✅ Ground truth generated using llma-8192 and saved to 'surah_baqarah_ground_truth.json'")

Processed: Who is Adam (A.S)?
Processed: What is the significance of the 'forbidden tree' in the story of Adam (AS)?
Processed: Why did the Jews of Madinah oppose Islam?
Processed: What does the Qur'an say about the hypocrites (Munafiqun)?
Processed: What lessons does the story of Adam (AS) teach about free will and repentance?
Processed: How does the Qur'an use the Israelites' history as a warning for Muslims?
✅ Ground truth generated using llma-8192 and saved to 'surah_baqarah_ground_truth_llama3.json'


In [15]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.2 MB/s eta 0:00:00


In [28]:
from transformers import logging
logging.set_verbosity_error()  # Add this before importing/using BERTScore

from bert_score import score
from bert_score import score
# Custom JSON encoder to handle numpy float32
class NumpyFloatValuesEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.float32):
            return float(obj)
        return super().default(obj)

# --- Set your Groq API key here ---
os.environ["GROQ_API_KEY"] = "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"

# Initialize cross-encoder for faithfulness scoring
cross_encoder = CrossEncoder('cross-encoder/stsb-roberta-large')

# Initialize ROUGE scorer for relevance
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# --- Helper Functions ---
def load_pdf(file_path):
    """Load PDF file and extract text."""
    text = ""
    with open(file_path, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

def load_ground_truth(file_path):
    """Load ground truth Q&A."""
    with open(file_path, "r") as f:
        return json.load(f)

def calculate_semantic_similarity(text1, text2, embedding_model):
    """Calculate cosine similarity between embeddings of two texts."""
    embedding1 = embedding_model.embed_query(text1)
    embedding2 = embedding_model.embed_query(text2)
    return float(cosine_similarity([embedding1], [embedding2])[0][0])  # Convert to Python float

def calculate_relevance_score(answer, context):
    """Calculate relevance using ROUGE scores between answer and context."""
    scores = rouge.score(answer, " ".join(context))
    return float((scores['rouge1'].fmeasure + scores['rouge2'].fmeasure + scores['rougeL'].fmeasure) / 3)
    # _, _, f1 = score([answer], [context], lang='en')
    # return float(f1.mean())

def calculate_faithfulness_score(answer, ground_truth):
    """Calculate faithfulness using cross-encoder between answer and ground truth."""
    return float(cross_encoder.predict([[answer, ground_truth]])[0])

def reciprocal_rank(relevant_doc_positions):
    """Calculate Reciprocal Rank (RR)."""
    if not relevant_doc_positions:
        return 0.0
    return float(1.0 / (relevant_doc_positions[0] + 1))  # Convert to Python float

In [29]:
# --- LLM Prompt Template ---
PROMPT_TEMPLATE = """
Answer the question based ONLY on the following context. If unsure, say "I don't know".

Context:
{context}

Question: {question}

Answer:
"""

class GroqLLM(BaseLLM):
    model_name: str = Field(default="meta-llama/llama-4-maverick-17b-128e-instruct")
    temperature: float = Field(default=0.3)
    max_tokens: int = Field(default=4000)

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._client = Groq(api_key=os.environ["GROQ_API_KEY"])

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> LLMResult:
        try:
            response = self._client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "user", "content": prompts[0]}],
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                stop=stop
            )
            return LLMResult(
                generations=[[{"text": response.choices[0].message.content}]]
            )
        except Exception as e:
            print(f"Error calling Groq API: {str(e)}")
            return LLMResult(generations=[[{"text": "I don't know"}]])

    @property
    def _llm_type(self) -> str:
        return "groq"

# --- RAG Pipeline Builder ---
def build_rag_pipeline(texts, config, embedding):
    """Build RAG pipeline with specified config."""
    # Vector Store
    if config["vector_store"] == "Chroma":
        # Clear existing Chroma collection when changing embedding models
        client = chromadb.PersistentClient(path="./chroma_db")
        try:
            # Delete existing collection if it exists
            client.delete_collection("rag_collection")
        except:
            pass  # Collection didn't exist

        # Create new collection with current embedding
        vectorstore = Chroma.from_texts(
            texts=texts,
            embedding=embedding,
            client=client,
            collection_name="rag_collection"
        )
    else:  # FAISS
        vectorstore = FAISS.from_texts(texts, embedding)

    # Retrievers
    if config["ranking"] == "BM25":
        retriever = BM25Retriever.from_texts(texts, k=4)
    elif config["ranking"] == "dense":
        retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
    else:  # Hybrid
        bm25_retriever = BM25Retriever.from_texts(texts, k=4)
        dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
        retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, dense_retriever],
            weights=[0.5, 0.5],
            c=60
        )

    prompt = PromptTemplate(
        template=PROMPT_TEMPLATE,
        input_variables=["context", "question"]
    )

    llm = GroqLLM(
        model_name=config["llm"],
        temperature=0.3,
        max_tokens=4000
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )
    return qa_chain

In [30]:
# --- Configuration Setup ---
CONFIGS = [
    # 500-100 chunks
    {
        "chunk_size": 500,
        "chunk_overlap": 100,
        "vector_store": "Chroma",

        "embedding": "sentence-transformers/all-MiniLM-L6-v2",
        "llm": "llama3-8b-8192",
        "ranking": "BM25",
        "search_type": "semantic",
        "processing": None
    },
    {
        "chunk_size": 500,
        "chunk_overlap": 100,
        "vector_store": "FAISS",

        "embedding": "sentence-transformers/all-MiniLM-L6-v2",
        "llm": "qwen-qwq-32b",
        "ranking": "dense",
        "search_type": "keyword",
        "processing": "summarization"
    },
    {
        "chunk_size": 500,
        "chunk_overlap": 100,
        "vector_store": "Chroma",

        "embedding": "sentence-transformers/all-MiniLM-L6-v2",
        "llm": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "ranking": "hybrid",
        "search_type": "semantic",
        "processing": "long_context_reordering"
    },
    {
        "chunk_size": 500,
        "chunk_overlap": 100,
        "vector_store": "FAISS",

        "embedding": "BAAI/bge-base-en",
        "llm": "llama3-8b-8192",
        "ranking": "BM25",
        "search_type": "semantic",
        "processing": None
    },
    {
        "chunk_size": 500,
        "chunk_overlap": 100,
        "vector_store": "Chroma",

        "embedding": "BAAI/bge-base-en",
        "llm": "qwen-qwq-32b",
        "ranking": "dense",
        "search_type": "keyword",
        "processing": "summarization"
    },
    {
        "chunk_size": 500,
        "chunk_overlap": 100,
        "vector_store": "FAISS",

        "embedding": "BAAI/bge-base-en",
        "llm": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "ranking": "hybrid",
        "search_type": "keyword",
        "processing": "long_context_reordering"
    },
    {
        "chunk_size": 500,
        "chunk_overlap": 100,
        "vector_store": "Chroma",

        "embedding": "intfloat/e5-base-v2",
        "llm": "llama3-8b-8192",
        "ranking": "BM25",
        "search_type": "semantic",
        "processing": None
    },
    {
        "chunk_size": 500,
        "chunk_overlap": 100,
        "vector_store": "FAISS",

        "embedding": "intfloat/e5-base-v2",
        "llm": "qwen-qwq-32b",
        "ranking": "dense",
        "search_type": "keyword",
        "processing": "summarization"
    },
    {
        "chunk_size": 500,
        "chunk_overlap": 100,
        "vector_store": "Chroma",

        "embedding": "intfloat/e5-base-v2",
        "llm": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "ranking": "hybrid",
        "search_type": "semantic",
        "processing": "long_context_reordering"
    },
]

# --- Evaluation Function ---
def evaluate_config(config, questions, ground_truths, embedding_model):
    """Evaluate a single RAG configuration."""
    text = load_pdf("/content/Surah Baqarah tafsir english (2).pdf")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=config["chunk_size"],
        chunk_overlap=config["chunk_overlap"]
    )
    texts = text_splitter.split_text(text)

    embedding = HuggingFaceEmbeddings(model_name=config["embedding"])

    start_time = time.time()
    qa_chain = build_rag_pipeline(texts, config, embedding)
    pipeline_time = time.time() - start_time

    results = []
    for q_idx, question in enumerate(questions):
        start_time = time.time()
        try:
            result = qa_chain.invoke({"query": question})
            time_taken = time.time() - start_time

            answer = result["result"]
            retrieved_docs = result["source_documents"][:4]
            doc_contents = [doc.page_content for doc in retrieved_docs]
            gt_answer = ground_truths[q_idx]["answer"]

            # Calculate all metrics
            semantic_sim = calculate_semantic_similarity(gt_answer, answer, embedding_model)
            relevance = calculate_relevance_score(answer, doc_contents)
            faithfulness = calculate_faithfulness_score(answer, gt_answer)

            results.append({
                "question": question,
                "ground_truth": gt_answer,
                "generated_answer": answer,
                "retrieved_docs": doc_contents,
                "time_taken": float(time_taken),
                "semantic_similarity": semantic_sim,
                "relevance_score": relevance,
                "faithfulness_score": faithfulness,
            })
        except Exception as e:
            print(f"Error processing question: {question}")
            print(f"Error: {str(e)}")
            continue

    # Aggregate metrics
    if results:
        avg_metrics = {
            "time_taken": float(np.mean([r["time_taken"] for r in results])),
            "semantic_similarity": float(np.mean([r["semantic_similarity"] for r in results])),
            "relevance_score": float(np.mean([r["relevance_score"] for r in results])),
            "faithfulness_score": float(np.mean([r["faithfulness_score"] for r in results])),
        }
    else:
        avg_metrics = {
            "time_taken": 0.0,
            "semantic_similarity": 0.0,
            "relevance_score": 0.0,
            "faithfulness_score": 0.0,
        }

    return {
        "config": config,
        "avg_metrics": avg_metrics,
        "results": results
    }

# --- Main Execution ---
# Load data once
ground_truths = load_ground_truth("surah_baqarah_ground_truth.json")
questions = [q["question"] for q in ground_truths]

# Run evaluations
all_results = []
for config in CONFIGS:
    print(f"\nTesting: {config}")
    embedding_model = HuggingFaceEmbeddings(model_name=config["embedding"])
    result = evaluate_config(config, questions, ground_truths, embedding_model)
    all_results.append(result)

    metrics = result["avg_metrics"]
    print(f"Results:")
    print(f"- Time: {metrics['time_taken']:.2f}s")
    print(f"- Semantic Similarity: {metrics['semantic_similarity']:.2f}")
    print(f"- Relevance: {metrics['relevance_score']:.2f}")
    print(f"- Faithfulness: {metrics['faithfulness_score']:.2f}")

# Save results
with open("rag_config_results_1.json", "w") as f:
    json.dump(all_results, f, indent=4, cls=NumpyFloatValuesEncoder)
print("\n✅ Results saved to 'rag_config_results.json'")


Testing: {'chunk_size': 500, 'chunk_overlap': 100, 'vector_store': 'Chroma', 'query_expansion': False, 'embedding': 'sentence-transformers/all-MiniLM-L6-v2', 'llm': 'llama3-8b-8192', 'ranking': 'BM25', 'search_type': 'semantic', 'processing': None}
Results:
- Time: 0.39s
- Semantic Similarity: 0.62
- Relevance: 0.12
- Faithfulness: 0.56

Testing: {'chunk_size': 500, 'chunk_overlap': 100, 'vector_store': 'FAISS', 'query_expansion': True, 'embedding': 'sentence-transformers/all-MiniLM-L6-v2', 'llm': 'qwen-qwq-32b', 'ranking': 'dense', 'search_type': 'keyword', 'processing': 'summarization'}
Results:
- Time: 1.58s
- Semantic Similarity: 0.72
- Relevance: 0.29
- Faithfulness: 0.62

Testing: {'chunk_size': 500, 'chunk_overlap': 100, 'vector_store': 'Chroma', 'query_expansion': True, 'embedding': 'sentence-transformers/all-MiniLM-L6-v2', 'llm': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'ranking': 'hybrid', 'search_type': 'semantic', 'processing': 'long_context_reordering'}
Results:
-

In [31]:
# --- Configuration Setup ---
CONFIGS = [
    # 1000-200 chunks
    {
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "vector_store": "Chroma",

        "embedding": "sentence-transformers/all-MiniLM-L6-v2",
        "llm": "llama3-8b-8192",
        "ranking": "BM25",
        "search_type": "semantic",
        "processing": None
    },
    {
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "vector_store": "FAISS",

        "embedding": "sentence-transformers/all-MiniLM-L6-v2",
        "llm": "qwen-qwq-32b",
        "ranking": "dense",
        "search_type": "keyword",
        "processing": "summarization"
    },
    {
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "vector_store": "Chroma",

        "embedding": "sentence-transformers/all-MiniLM-L6-v2",
        "llm": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "ranking": "hybrid",
        "search_type": "semantic",
        "processing": "long_context_reordering"
    },
    {
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "vector_store": "FAISS",

        "embedding": "BAAI/bge-base-en",
        "llm": "llama3-8b-8192",
        "ranking": "BM25",
        "search_type": "semantic",
        "processing": None
    },
    {
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "vector_store": "Chroma",

        "embedding": "BAAI/bge-base-en",
        "llm": "qwen-qwq-32b",
        "ranking": "dense",
        "search_type": "keyword",
        "processing": "summarization"
    },
    {
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "vector_store": "FAISS",

        "embedding": "BAAI/bge-base-en",
        "llm": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "ranking": "hybrid",
        "search_type": "keyword",
        "processing": "long_context_reordering"
    },
    {
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "vector_store": "Chroma",

        "embedding": "intfloat/e5-base-v2",
        "llm": "llama3-8b-8192",
        "ranking": "BM25",
        "search_type": "semantic",
        "processing": None
    },
    {
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "vector_store": "FAISS",

        "embedding": "intfloat/e5-base-v2",
        "llm": "qwen-qwq-32b",
        "ranking": "dense",
        "search_type": "keyword",
        "processing": "summarization"
    },
    {
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "vector_store": "Chroma",

        "embedding": "intfloat/e5-base-v2",
        "llm": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "ranking": "hybrid",
        "search_type": "semantic",
        "processing": "long_context_reordering"
    },
]

# --- Evaluation Function ---
def evaluate_config(config, questions, ground_truths, embedding_model):
    """Evaluate a single RAG configuration."""
    text = load_pdf("/content/Surah Baqarah tafsir english (2).pdf")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=config["chunk_size"],
        chunk_overlap=config["chunk_overlap"]
    )
    texts = text_splitter.split_text(text)

    embedding = HuggingFaceEmbeddings(model_name=config["embedding"])

    start_time = time.time()
    qa_chain = build_rag_pipeline(texts, config, embedding)
    pipeline_time = time.time() - start_time

    results = []
    for q_idx, question in enumerate(questions):
        start_time = time.time()
        try:
            result = qa_chain.invoke({"query": question})
            time_taken = time.time() - start_time

            answer = result["result"]
            retrieved_docs = result["source_documents"][:4]
            doc_contents = [doc.page_content for doc in retrieved_docs]
            gt_answer = ground_truths[q_idx]["answer"]

            # Calculate all metrics
            semantic_sim = calculate_semantic_similarity(gt_answer, answer, embedding_model)
            relevance = calculate_relevance_score(answer, doc_contents)
            faithfulness = calculate_faithfulness_score(answer, gt_answer)

            results.append({
                "question": question,
                "ground_truth": gt_answer,
                "generated_answer": answer,
                "retrieved_docs": doc_contents,
                "time_taken": float(time_taken),
                "semantic_similarity": semantic_sim,
                "relevance_score": relevance,
                "faithfulness_score": faithfulness,
            })
        except Exception as e:
            print(f"Error processing question: {question}")
            print(f"Error: {str(e)}")
            continue

    # Aggregate metrics
    if results:
        avg_metrics = {
            "time_taken": float(np.mean([r["time_taken"] for r in results])),
            "semantic_similarity": float(np.mean([r["semantic_similarity"] for r in results])),
            "relevance_score": float(np.mean([r["relevance_score"] for r in results])),
            "faithfulness_score": float(np.mean([r["faithfulness_score"] for r in results])),
        }
    else:
        avg_metrics = {
            "time_taken": 0.0,
            "semantic_similarity": 0.0,
            "relevance_score": 0.0,
            "faithfulness_score": 0.0,
        }

    return {
        "config": config,
        "avg_metrics": avg_metrics,
        "results": results
    }

# --- Main Execution ---
# Load data once
ground_truths = load_ground_truth("surah_baqarah_ground_truth.json")
questions = [q["question"] for q in ground_truths]

# Run evaluations
all_results = []
for config in CONFIGS:
    print(f"\nTesting: {config}")
    embedding_model = HuggingFaceEmbeddings(model_name=config["embedding"])
    result = evaluate_config(config, questions, ground_truths, embedding_model)
    all_results.append(result)

    metrics = result["avg_metrics"]
    print(f"Results:")
    print(f"- Time: {metrics['time_taken']:.2f}s")
    print(f"- Semantic Similarity: {metrics['semantic_similarity']:.2f}")
    print(f"- Relevance: {metrics['relevance_score']:.2f}")
    print(f"- Faithfulness: {metrics['faithfulness_score']:.2f}")

# Save results
with open("rag_config_results_2.json", "w") as f:
    json.dump(all_results, f, indent=4, cls=NumpyFloatValuesEncoder)
print("\n✅ Results saved to 'rag_config_results.json'")


Testing: {'chunk_size': 1000, 'chunk_overlap': 200, 'vector_store': 'Chroma', 'query_expansion': False, 'embedding': 'sentence-transformers/all-MiniLM-L6-v2', 'llm': 'llama3-8b-8192', 'ranking': 'BM25', 'search_type': 'semantic', 'processing': None}
Results:
- Time: 0.71s
- Semantic Similarity: 0.67
- Relevance: 0.08
- Faithfulness: 0.56

Testing: {'chunk_size': 1000, 'chunk_overlap': 200, 'vector_store': 'FAISS', 'query_expansion': True, 'embedding': 'sentence-transformers/all-MiniLM-L6-v2', 'llm': 'qwen-qwq-32b', 'ranking': 'dense', 'search_type': 'keyword', 'processing': 'summarization'}
Results:
- Time: 4.78s
- Semantic Similarity: 0.71
- Relevance: 0.26
- Faithfulness: 0.66

Testing: {'chunk_size': 1000, 'chunk_overlap': 200, 'vector_store': 'Chroma', 'query_expansion': True, 'embedding': 'sentence-transformers/all-MiniLM-L6-v2', 'llm': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'ranking': 'hybrid', 'search_type': 'semantic', 'processing': 'long_context_reordering'}
Results

In [32]:
# --- Configuration Setup ---
CONFIGS = [
    # 1500-300 chunks
    {
        "chunk_size": 1500,
        "chunk_overlap": 300,
        "vector_store": "Chroma",

        "embedding": "sentence-transformers/all-MiniLM-L6-v2",
        "llm": "llama3-8b-8192",
        "ranking": "BM25",
        "search_type": "semantic",
        "processing": None
    },
    {
        "chunk_size": 1500,
        "chunk_overlap": 300,
        "vector_store": "FAISS",

        "embedding": "sentence-transformers/all-MiniLM-L6-v2",
        "llm": "qwen-qwq-32b",
        "ranking": "dense",
        "search_type": "keyword",
        "processing": "summarization"
    },
    {
        "chunk_size": 1500,
        "chunk_overlap": 300,
        "vector_store": "Chroma",

        "embedding": "sentence-transformers/all-MiniLM-L6-v2",
        "llm": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "ranking": "hybrid",
        "search_type": "semantic",
        "processing": "long_context_reordering"
    },
    {
        "chunk_size": 1500,
        "chunk_overlap": 300,
        "vector_store": "FAISS",

        "embedding": "BAAI/bge-base-en",
        "llm": "llama3-8b-8192",
        "ranking": "BM25",
        "search_type": "semantic",
        "processing": None
    },
    {
        "chunk_size": 1500,
        "chunk_overlap": 300,
        "vector_store": "Chroma",

        "embedding": "BAAI/bge-base-en",
        "llm": "qwen-qwq-32b",
        "ranking": "dense",
        "search_type": "keyword",
        "processing": "summarization"
    },
    {
        "chunk_size": 1500,
        "chunk_overlap": 300,
        "vector_store": "FAISS",

        "embedding": "BAAI/bge-base-en",
        "llm": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "ranking": "hybrid",
        "search_type": "keyword",
        "processing": "long_context_reordering"
    },
    {
        "chunk_size": 1500,
        "chunk_overlap": 300,
        "vector_store": "Chroma",

        "embedding": "intfloat/e5-base-v2",
        "llm": "llama3-8b-8192",
        "ranking": "BM25",
        "search_type": "semantic",
        "processing": None
    },
    {
        "chunk_size": 1500,
        "chunk_overlap": 300,
        "vector_store": "FAISS",

        "embedding": "intfloat/e5-base-v2",
        "llm": "qwen-qwq-32b",
        "ranking": "dense",
        "search_type": "keyword",
        "processing": "summarization"
    },
    {
        "chunk_size": 1500,
        "chunk_overlap": 300,
        "vector_store": "Chroma",

        "embedding": "intfloat/e5-base-v2",
        "llm": "meta-llama/llama-4-maverick-17b-128e-instruct",
        "ranking": "hybrid",
        "search_type": "semantic",
        "processing": "long_context_reordering"
    },
]

# --- Evaluation Function ---
def evaluate_config(config, questions, ground_truths, embedding_model):
    """Evaluate a single RAG configuration."""
    text = load_pdf("/content/Surah Baqarah tafsir english (2).pdf")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=config["chunk_size"],
        chunk_overlap=config["chunk_overlap"]
    )
    texts = text_splitter.split_text(text)

    embedding = HuggingFaceEmbeddings(model_name=config["embedding"])

    start_time = time.time()
    qa_chain = build_rag_pipeline(texts, config, embedding)
    pipeline_time = time.time() - start_time

    results = []
    for q_idx, question in enumerate(questions):
        start_time = time.time()
        try:
            result = qa_chain.invoke({"query": question})
            time_taken = time.time() - start_time

            answer = result["result"]
            retrieved_docs = result["source_documents"][:4]
            doc_contents = [doc.page_content for doc in retrieved_docs]
            gt_answer = ground_truths[q_idx]["answer"]

            # Calculate all metrics
            semantic_sim = calculate_semantic_similarity(gt_answer, answer, embedding_model)
            relevance = calculate_relevance_score(answer, doc_contents)
            faithfulness = calculate_faithfulness_score(answer, gt_answer)

            results.append({
                "question": question,
                "ground_truth": gt_answer,
                "generated_answer": answer,
                "retrieved_docs": doc_contents,
                "time_taken": float(time_taken),
                "semantic_similarity": semantic_sim,
                "relevance_score": relevance,
                "faithfulness_score": faithfulness,
            })
        except Exception as e:
            print(f"Error processing question: {question}")
            print(f"Error: {str(e)}")
            continue

    # Aggregate metrics
    if results:
        avg_metrics = {
            "time_taken": float(np.mean([r["time_taken"] for r in results])),
            "semantic_similarity": float(np.mean([r["semantic_similarity"] for r in results])),
            "relevance_score": float(np.mean([r["relevance_score"] for r in results])),
            "faithfulness_score": float(np.mean([r["faithfulness_score"] for r in results])),
        }
    else:
        avg_metrics = {
            "time_taken": 0.0,
            "semantic_similarity": 0.0,
            "relevance_score": 0.0,
            "faithfulness_score": 0.0,
        }

    return {
        "config": config,
        "avg_metrics": avg_metrics,
        "results": results
    }

# --- Main Execution ---
# Load data once
ground_truths = load_ground_truth("surah_baqarah_ground_truth.json")
questions = [q["question"] for q in ground_truths]

# Run evaluations
all_results = []
for config in CONFIGS:
    print(f"\nTesting: {config}")
    embedding_model = HuggingFaceEmbeddings(model_name=config["embedding"])
    result = evaluate_config(config, questions, ground_truths, embedding_model)
    all_results.append(result)

    metrics = result["avg_metrics"]
    print(f"Results:")
    print(f"- Time: {metrics['time_taken']:.2f}s")
    print(f"- Semantic Similarity: {metrics['semantic_similarity']:.2f}")
    print(f"- Relevance: {metrics['relevance_score']:.2f}")
    print(f"- Faithfulness: {metrics['faithfulness_score']:.2f}")

# Save results
with open("rag_config_results_3.json", "w") as f:
    json.dump(all_results, f, indent=4, cls=NumpyFloatValuesEncoder)
print("\n✅ Results saved to 'rag_config_results.json'")


Testing: {'chunk_size': 1500, 'chunk_overlap': 300, 'vector_store': 'Chroma', 'query_expansion': False, 'embedding': 'sentence-transformers/all-MiniLM-L6-v2', 'llm': 'llama3-8b-8192', 'ranking': 'BM25', 'search_type': 'semantic', 'processing': None}
Results:
- Time: 4.71s
- Semantic Similarity: 0.72
- Relevance: 0.07
- Faithfulness: 0.58

Testing: {'chunk_size': 1500, 'chunk_overlap': 300, 'vector_store': 'FAISS', 'query_expansion': True, 'embedding': 'sentence-transformers/all-MiniLM-L6-v2', 'llm': 'qwen-qwq-32b', 'ranking': 'dense', 'search_type': 'keyword', 'processing': 'summarization'}
Results:
- Time: 3.53s
- Semantic Similarity: 0.69
- Relevance: 0.24
- Faithfulness: 0.62

Testing: {'chunk_size': 1500, 'chunk_overlap': 300, 'vector_store': 'Chroma', 'query_expansion': True, 'embedding': 'sentence-transformers/all-MiniLM-L6-v2', 'llm': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'ranking': 'hybrid', 'search_type': 'semantic', 'processing': 'long_context_reordering'}
Results

### After evaluating all the models (explaination in pdf report/json/excel), the below configuration is selected for our final RAG pipeline

In [65]:
# --- Configuration ---
config = {
    "chunk_size": 500,
    "chunk_overlap": 100,
    "vector_store": "Chroma",
    "embedding": "BAAI/bge-base-en",
    "llm": "qwen-qwq-32b",
    "ranking": "dense",
    "search_type": "keyword",
    "processing": "summarization"
}

os.environ["GROQ_API_KEY"] = "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"

In [66]:
# --- LLM Implementation ---
class CustomLLM(BaseLLM):
    model_name: str = Field(default="qwen-qwq-32b")
    temperature: float = Field(default=0.3)
    max_tokens: int = Field(default=4000)

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # Initialize with either Groq or HuggingFace based on availability
        if "GROQ_API_KEY" in os.environ:
            self._client = Groq(api_key=os.environ["GROQ_API_KEY"])
            self._use_groq = True
        else:
            print("Need API Key")

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> LLMResult:
        try:
            if self._use_groq:
                response = self._client.chat.completions.create(
                    model=self.model_name,
                    messages=[{"role": "user", "content": prompts[0]}],
                    temperature=self.temperature,
                    max_tokens=self.max_tokens,
                    stop=stop
                )
                text = response.choices[0].message.content
            else:
                text = self._client(prompts[0])

            return LLMResult(generations=[[{"text": text}]])
        except Exception as e:
            print(f"Error calling LLM API: {str(e)}")
            return LLMResult(generations=[[{"text": "I don't know"}]])

    @property
    def _llm_type(self) -> str:
        return "custom_llm"

# --- Prompt Template ---
PROMPT_TEMPLATE = """
Answer the question based ONLY on the following context. If unsure, say "I don't know".

Context:
{context}

Question: {question}

Answer:
"""

# --- Document Processing ---
def load_and_split_documents(file_path):
    """Load PDF and split documents according to config."""
    # Load PDF file
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"

    # Split text according to config
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=config["chunk_size"],
        chunk_overlap=config["chunk_overlap"]
    )
    return text_splitter.split_text(text)


# --- Updated Test Pipeline ---
def test_pipeline(pipeline, questions):
    """Test the pipeline with sample questions."""
    results = []
    for question in questions:
        print(f"\nQuestion: {question}")

        # Get answer
        start_time = time.time()
        try:
            result = pipeline.invoke({"query": question})
            time_taken = time.time() - start_time

            # Display results
            print(f"Answer: {result['result']}")
            print(f"Time taken: {time_taken:.2f}s")

            # Show sources if available
            if "source_documents" in result:
                print("\nTop Sources:")
                for i, doc in enumerate(result["source_documents"][:3]):
                    print(f"{i+1}. {doc.page_content[:200]}..." if len(doc.page_content) > 200 else f"{i+1}. {doc.page_content}")

            results.append({
                "question": question,
                "answer": result["result"],
                "time_taken": time_taken,
                "sources": [doc.page_content for doc in result.get("source_documents", [])]
            })
        except Exception as e:
            print(f"Error processing question: {str(e)}")
            results.append({
                "question": question,
                "error": str(e)
            })

    return results

# --- Pipeline Builder ---
def build_pipeline(documents):
    """Build the complete RAG pipeline."""
    # Initialize embedding
    embedding = HuggingFaceEmbeddings(model_name=config["embedding"])

    # Create vector store
    if config["vector_store"] == "Chroma":
        client = chromadb.PersistentClient(path="./chroma_db")
        try:
            client.delete_collection("rag_collection")
        except:
            pass

        vectorstore = Chroma.from_texts(
            texts=documents,
            embedding=embedding,
            client=client,
            collection_name="rag_collection"
        )
    else:
        vectorstore = FAISS.from_texts(documents, embedding)

    # Create retriever based on ranking strategy
    if config["ranking"] == "BM25":
        retriever = BM25Retriever.from_texts(documents, k=4)
    elif config["ranking"] == "dense":
        retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
    else:  # Hybrid
        bm25_retriever = BM25Retriever.from_texts(documents, k=4)
        dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
        retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, dense_retriever],
            weights=[0.5, 0.5]
        )

    # Initialize LLM
    llm = CustomLLM(
        model_name=config["llm"],
        temperature=0.3,
        max_tokens=4000
    )

    # Create prompt template
    prompt = PromptTemplate(
        template=PROMPT_TEMPLATE,
        input_variables=["context", "question"]
    )

    # Build QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    return qa_chain

In [67]:
# --- Main Execution ---
if __name__ == "__main__":
    # Sample document and questions
    document_path = "/content/Surah Baqarah tafsir english (2).pdf"
    sample_questions = [
        "Explain the significance of the cow",
        "What are the key lessons from the story of Musa?"
    ]

    # Build and test pipeline
    print("Loading and processing documents...")
    try:
        documents = load_and_split_documents(document_path)
        print(f"Loaded {len(documents)} document chunks")

        print("\nBuilding pipeline...")
        pipeline = build_pipeline(documents)

        print("\nTesting pipeline with sample questions...")
        test_results = test_pipeline(pipeline, sample_questions)

        print("\nPipeline testing complete!")

        # Print summary of results
        print("\nTest Results Summary:")
        for i, result in enumerate(test_results):
            print(f"\nQuestion {i+1}: {result['question']}")
            if 'answer' in result:
                print(f"Answer: {result['answer'][:200]}..." if len(result['answer']) > 200 else f"Answer: {result['answer']}")
                print(f"Time: {result['time_taken']:.2f}s")
            else:
                print(f"Error: {result['error']}")

    except Exception as e:
        print(f"Pipeline failed: {str(e)}")

Loading and processing documents...
Loaded 1838 document chunks

Building pipeline...

Testing pipeline with sample questions...

Question: Explain the significance of the cow
Answer: 
<think>
Okay, I need to explain the significance of the cow based on the provided context. Let me read through the context again to make sure I understand all the points mentioned.

The context starts by describing the cow as bright yellow, not too fat or lean, and must be pleasing to behold—lively and vigorous. Then it talks about Jews making things difficult for themselves by asking about the type of cow they should look for. There's mention of metaphorical language in the Quran about the calf, showing strong love and adoration, which is part of the Quranic style. The Jews are portrayed as God's chosen people but make a big fuss, and there's a reference to them being turned into apes, which was a spiritual state. The main story here is the Episode of the Cow, which is a full account in the Quran. The s